# Installation

There are some libraries that we need and with which the code was tested

* **taxii2-client**: To talk to the taxii server
* **stix2**: To handle the stix2 encapsulated data
* **networkx**: To create and work with graphs
* **bokeh**: To visualize graphs

In [2]:
%pip install taxii2-client==2.3.0
%pip install stix2==3.0.1
%pip install networkx==2.6.3
%pip install bokeh==2.3.3
%pip install pandas==1.3.5

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Setup

We lower the debugging level to only show critical messages.
We use the following URL to work with:
```https://cti-taxii.mitre.org/taxii/```

In [3]:
from taxii2client.v20 import Server

import logging
logging.getLogger('taxii2client').setLevel(logging.CRITICAL)
server = Server("https://cti-taxii.mitre.org/taxii/")

# Retrieve all the data

Let's get all the available collections and select the collection that we want to work with

In [4]:
api_root = server.api_roots[0]
for idx, collection in enumerate(api_root.collections):
    print(f"[{idx}] {collection.title} -> {collection.description}")

[0] Enterprise ATT&CK -> This data collection holds STIX objects from Enterprise ATT&CK
[1] PRE-ATT&CK -> This data collection holds STIX objects from PRE-ATT&CK
[2] Mobile ATT&CK -> This data collection holds STIX objects from Mobile ATT&CK
[3] ICS ATT&CK -> This data collection holds STIX objects from ICS ATT&CK


Make a conscious decision on the collection we want to work with

In [5]:
MITRE_COLLECTION_ID = api_root.collections[3].id

For the selected collection, actually get the te collection

In [6]:
from stix2 import TAXIICollectionSource, Filter
from taxii2client.v20 import Collection

ATTACK_STIX_COLLECTIONS = "https://cti-taxii.mitre.org/stix/collections/"
MITRE_COLLECTION = Collection(ATTACK_STIX_COLLECTIONS + MITRE_COLLECTION_ID + "/")
MC_DATA = TAXIICollectionSource(MITRE_COLLECTION)

Also get the mitigation (course of action), detections and technique data. We retreive it as a mapping of unique IDs and human readable names

In [7]:
MC_MITIGATIONS = MC_DATA.query(Filter("type", "=", "course-of-action"))
mc_mitigation_mapping = {}
for x in MC_MITIGATIONS:
  mc_mitigation_mapping[x['id']] = x['name']

print(mc_mitigation_mapping)

{'course-of-action--1cbcceef-3233-4062-aa86-ec91afe39517': 'Validate Program Inputs', 'course-of-action--ac8f3492-7fbb-4a0a-b0b4-b75ec676136c': 'Supply Chain Management', 'course-of-action--469b78dd-a54d-4f7c-8c3b-4a1dd916b433': 'Mitigation Limited or Not Effective', 'course-of-action--8bc4a54e-810c-4600-8b6c-08fa8413a401': 'Mechanical Protection Layers', 'course-of-action--fce6866f-9a87-4d3e-a73c-f02d8937fe0e': 'Minimize Wireless Signal Propagation', 'course-of-action--66cfe23e-34b6-4583-b178-ed6a412db2b0': 'Human User Authentication', 'course-of-action--c7257b6e-4159-4771-b1f3-2bb93adaecac': 'Communication Authenticity', 'course-of-action--7f153c28-e5f1-4764-88fb-eea1d9b0ad4a': 'Encrypt Network Traffic', 'course-of-action--337c4e2a-21a7-4d9a-bfee-9efd6cebf0e5': 'Data Loss Prevention', 'course-of-action--3992ce42-43e9-4bea-b8db-a102ec3ec1e3': 'Access Management', 'course-of-action--e0d38502-decb-481d-ad8b-b8f0a0c330bd': 'Authorization Enforcement', 'course-of-action--facb8840-ebe7-49f

In [8]:
MC_TECHNIQUES = MC_DATA.query(Filter("type", "=", "attack-pattern"))
mc_technique_mapping = {}
for x in MC_TECHNIQUES:
  mc_technique_mapping[x['id']] = x['name']

In [9]:
MC_DETECTIONS = MC_DATA.query(Filter("type", "=", "x-mitre-data-component"))
mc_detection_mapping = {}
for x in MC_DETECTIONS:
  mc_detection_mapping[x['id']] = x['name']

In [9]:
MC_INTRUSION_SET = MC_DATA.query(Filter("type", "=", "intrusion-set"))
mc_intrusion_set_mapping = {}
for x in MC_INTRUSION_SET:
  mc_intrusion_set_mapping[x['id']] = x['name']
print(mc_intrusion_set_mapping)

{'intrusion-set--c77c5576-ca19-42ed-a36f-4b4486a84133': 'GOLD SOUTHFIELD', 'intrusion-set--dd2d9ca6-505b-4860-a604-233685b802c7': 'Wizard Spider', 'intrusion-set--9538b1a4-4120-4e2d-bf59-3b11fcab05a4': 'TEMP.Veles', 'intrusion-set--00f67a77-86a4-4adf-be26-1a54fc713340': 'APT38', 'intrusion-set--76d59913-1d24-4992-a8ac-05a3eb093f71': 'Dragonfly 2.0', 'intrusion-set--f29b7c5e-2439-42ad-a86f-9f8984fafae3': 'HEXANE', 'intrusion-set--fbd29c89-18ba-4c2d-b792-51c0adee049f': 'APT33', 'intrusion-set--68ba94ab-78b8-43e7-83e2-aed3466882c6': 'APT34', 'intrusion-set--4ca1929c-7d64-4aab-b849-badbfc0c760d': 'OilRig', 'intrusion-set--3753cc21-2dae-4dfb-8481-d004e74502cc': 'FIN7', 'intrusion-set--2a7914cf-dff3-428d-ab0f-1014d1c28aeb': 'FIN6', 'intrusion-set--1c63d4ec-0a75-4daa-b1df-0d11af3d3cc1': 'Dragonfly', 'intrusion-set--381fcf73-60f6-4ab2-9991-6af3cbc35192': 'Sandworm Team', 'intrusion-set--c93fccb1-e8e8-42cf-ae33-2ad1d183913a': 'Lazarus Group', 'intrusion-set--190242d7-73fc-4738-af68-20162f7a5aae

In [12]:
MC_MALWARE = MC_DATA.query(Filter("type", "=", "malware"))
mc_malware_mapping = {}
for x in MC_MALWARE:
  mc_malware_mapping[x['id']] = x['name']

Let's now get the relationship between course of actions (coa), detection sources and techniques. We also put the data into a panda dataframe and just for fun we ad the human readable names as well.

In [18]:
import pandas as pd

MC_RELATIONS = MC_DATA.query(Filter("type","=","relationship"))
print(MC_RELATIONS)

mc_structured_data = {'Source':[],'Destination':[],'source_human':[],'target_human':[]}
for idx,x in enumerate(MC_RELATIONS):
  if 'relationship_type' in x:
    if x['relationship_type'] == 'mitigates':
      mc_structured_data['Source'].append(x['source_ref'])
      mc_structured_data['Destination'].append(x['target_ref'])
      mc_structured_data['source_human'].append(mc_mitigation_mapping[x['source_ref']])
      mc_structured_data['target_human'].append(mc_technique_mapping[x['target_ref']])
    elif x['relationship_type'] == 'detects':
      mc_structured_data['Source'].append(x['source_ref'])
      mc_structured_data['Destination'].append(x['target_ref'])
      mc_structured_data['source_human'].append(mc_detection_mapping[x['source_ref']])
      mc_structured_data['target_human'].append(mc_technique_mapping[x['target_ref']])


mc_ot_data = {'Source':[],'Destination':[],'source_human':[],'target_human':[]}
for idx,x in enumerate(MC_RELATIONS):
  if 'relationship_type' in x:
    if x['relationship_type'] == 'uses' and 'intrusion-set' in x['source_ref'] and 'malware' in x['target_ref']:
      mc_ot_data['Source'].append(x['source_ref'])
      mc_ot_data['Destination'].append(x['target_ref'])
      mc_ot_data['source_human'].append(mc_intrusion_set_mapping[x['source_ref']])
      mc_ot_data['target_human'].append(mc_malware_mapping[x['target_ref']])
    elif x['relationship_type'] == 'uses' and 'malware' in x['source_ref'] and 'attack-pattern' in x['target_ref']:
      mc_ot_data['Source'].append(x['source_ref'])
      mc_ot_data['Destination'].append(x['target_ref'])
      mc_ot_data['source_human'].append(mc_malware_mapping[x['source_ref']])
      mc_ot_data['target_human'].append(mc_technique_mapping[x['target_ref']])
    elif x['relationship_type'] == 'detects':
      mc_ot_data['Source'].append(x['source_ref'])
      mc_ot_data['Destination'].append(x['target_ref'])
      mc_ot_data['source_human'].append(mc_detection_mapping[x['source_ref']])
      mc_ot_data['target_human'].append(mc_technique_mapping[x['target_ref']])
      

df = pd.DataFrame(data=mc_ot_data)
df.head()
df.tail()

[Relationship(type='relationship', id='relationship--cbee31a0-716c-4b10-83f0-aa889bfb4749', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2023-10-20T17:05:25.595Z', modified='2023-10-20T17:05:25.595Z', relationship_type='targets', description='', source_ref='attack-pattern--097924ce-a9a9-4039-8591-e0deedfb8722', target_ref='x-mitre-asset--986c455b-0f43-42b6-8360-33ac48bd9990', revoked=False, object_marking_refs=['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'], x_mitre_attack_spec_version='3.2.0', x_mitre_deprecated=False, x_mitre_modified_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', x_mitre_version='0.1'), Relationship(type='relationship', id='relationship--1429cd78-4e2a-4898-a7d8-d01a0c465bd6', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2023-10-02T20:24:12.666Z', modified='2023-10-02T20:24:12.666Z', relationship_type='targets', description='', source_ref='attack-pattern--0fe075d5-beac-4d02-b93e-0f874997

,Source,Destination,source_human,target_human
395,malware--2eaa5319-5e1e-4dd7-bbc4-566fced3964a,attack-pattern--7830cfcf-b268-4ac0-a69e-73c6af...,Bad Rabbit,Drive-by Compromise
396,malware--2eaa5319-5e1e-4dd7-bbc4-566fced3964a,attack-pattern--63b6942d-8359-4506-bfb3-cf87aa...,Bad Rabbit,Loss of Productivity and Revenue
397,malware--2eaa5319-5e1e-4dd7-bbc4-566fced3964a,attack-pattern--2736b752-4ec5-4421-a230-8977de...,Bad Rabbit,User Execution
398,intrusion-set--381fcf73-60f6-4ab2-9991-6af3cbc...,malware--54cc1d4f-5c53-4f0e-9ef5-11b4998e82e4,Sandworm Team,BlackEnergy
399,intrusion-set--1c63d4ec-0a75-4daa-b1df-0d11af3...,malware--083bb47b-02c8-4423-81a2-f9ef58572974,Dragonfly,Backdoor.Oldrea


## Malware to attack-pattern relationship

 
Relationship(type='relationship', id='relationship--2087b2b9-3b30-45be-abcd-4320bf0fa66b', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2023-03-30T19:26:19.782Z', modified='2023-04-06T22:09:28.674Z', relationship_type='uses', description='[Industroyer2](https://attack.mitre.org/software/S1072) can iterate across a device’s IOAs to modify the ON/OFF value of a given IO state.(Citation: Industroyer2 Mandiant April 2022)(Citation: Industroyer2 Forescout July 2022)', source_ref='malware--6a0d0ea9-b2c4-43fe-a552-ac41a3009dc5', target_ref='attack-pattern--8e7089d3-fba2-44f8-94a8-9a79c53920c4', revoked=False, external_references=[ExternalReference(source_name='Industroyer2 Mandiant April 2022', description='Daniel Kapellmann Zafra, Raymond Leong, Chris Sistrunk, Ken Proska, Corey Hildebrandt, Keith Lunden, Nathan Brubaker. (2022, April 25). INDUSTROYER.V2: Old Malware Learns New Tricks. Retrieved March 30, 2023.', url='https://www.mandiant.com/resources/blog/industroyer-v2-old-malware-new-tricks'), ExternalReference(source_name='Industroyer2 Forescout July 2022', description='Forescout. (2022, July 14). Industroyer2 and  INCONTROLLER In-depth Technical Analysis of the Most Recent ICS-specific Malware. Retrieved March 30, 2023.', url='https://www.forescout.com/resources/industroyer2-and-incontroller-report/')], object_marking_refs=['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'], x_mitre_attack_spec_version='3.1.0', x_mitre_deprecated=False, x_mitre_modified_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', x_mitre_version='0.1'),


# Visualizing all the data

A lot of thanks to this information resource ```https://melaniewalsh.github.io/Intro-Cultural-Analytics/06-Network-Analysis/02-Making-Network-Viz-with-Bokeh.html``` excellent explanation and walkthrough on visualizing graph data with bokeh.

In [10]:
# https://melaniewalsh.github.io/Intro-Cultural-Analytics/06-Network-Analysis/02-Making-Network-Viz-with-Bokeh.html
from bokeh.io import output_notebook, show, save
# required to display visuals inline
output_notebook()

Loading BokehJS ...

In [28]:

from bokeh.core.property_mixins import LineJoin
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap
from bokeh.models import EdgesAndLinkedNodes, NodesAndLinkedEdges
import networkx as nx


DG = nx.from_pandas_edgelist(df, source='Source', target='Destination',create_using=nx.Graph())

# set the degrees (connections per node)
degrees = dict(nx.degree(DG))
nx.set_node_attributes(DG, name='degree', values=degrees)

#set the human readable attribute
all_mappings = {}
all_mappings.update(mc_technique_mapping)
all_mappings.update(mc_mitigation_mapping)
all_mappings.update(mc_detection_mapping)
all_mappings.update(mc_intrusion_set_mapping)
all_mappings.update(mc_malware_mapping)

nx.set_node_attributes(DG,name='human_name',values=all_mappings)

#ensure very small degrees are still visible
number_to_adjust_by = 5
adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in nx.degree(DG)])
nx.set_node_attributes(DG, name='adjusted_node_size', values=adjusted_node_size)

# determine type, add attribute
node_types = {}
# for node in DG.nodes:
#   if node.startswith('attack-pattern'):
#     node_types[node] = 0
#   elif node.startswith('course-of-action'):
#     node_types[node] = 1
#   else:
#     node_types[node] = 2

for node in DG.nodes:
    if node.startswith('intrusion-set'):
      node_types[node] = 0
    elif node.startswith('malware'):
      node_types[node] = 1 
    elif node.startswith('attack-pattern'):
      node_types[node] = 2
    else:
      node_types[node] = 3

#set the node type (attack-pattern,course-of-action,data-source)
nx.set_node_attributes(DG, name='node_type', values=node_types)

#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'node_type'

#color_palette = red,green,blue
color_palette = ['#A020F0','#FF0000','#00FF00','#0000FF']

#Choose colors for node and edge highlighting
node_highlight_color = 'white'
edge_highlight_color = 'black'

#Choose a title!
title = 'MITRE Graph Visuals'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [("Name", "@human_name"),("Degree", "@degree")]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object with spring layout
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
network_graph = from_networkx(DG, nx.spring_layout, scale=10, center=(0, 0))
#network_graph = from_networkx(DG, nx.kamada_kawai_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as spectrum of color palette)
minimum_value_color = 0
maximum_value_color = 2
#Set node size and color
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=linear_cmap(color_by_this_attribute, color_palette, minimum_value_color, maximum_value_color))

#Set node highlight colors
network_graph.node_renderer.hover_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)
network_graph.node_renderer.selection_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)


#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

#Set edge highlight colors
network_graph.edge_renderer.selection_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)
network_graph.edge_renderer.hover_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)

#Highlight nodes and edges
network_graph.selection_policy = NodesAndLinkedEdges()
network_graph.inspection_policy = NodesAndLinkedEdges()


#Add network graph to the plot
plot.renderers.append(network_graph)
plot.plot_width = 1100
plot.plot_height = 550
show(plot)
#save(plot, filename=f"{title}.html")

Opening in existing browser session.


In [11]:
from bokeh.core.property_mixins import LineJoin
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap
from bokeh.models import EdgesAndLinkedNodes, NodesAndLinkedEdges
import networkx as nx


DG = nx.from_pandas_edgelist(df, source='Source', target='Destination',create_using=nx.Graph())

# set the degrees (connections per node)
degrees = dict(nx.degree(DG))
nx.set_node_attributes(DG, name='degree', values=degrees)

#set the human readable attribute
all_mappings = {}
all_mappings.update(mc_technique_mapping)
all_mappings.update(mc_mitigation_mapping)
all_mappings.update(mc_detection_mapping)
nx.set_node_attributes(DG,name='human_name',values=all_mappings)

#ensure very small degrees are still visible
number_to_adjust_by = 5
adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in nx.degree(DG)])
nx.set_node_attributes(DG, name='adjusted_node_size', values=adjusted_node_size)

# determine type, add attribute
node_types = {}
for node in DG.nodes:
  if node.startswith('attack-pattern'):
    node_types[node] = 0
  elif node.startswith('course-of-action'):
    node_types[node] = 1
  else:
    node_types[node] = 2

#set the node type (attack-pattern,course-of-action,data-source)
nx.set_node_attributes(DG, name='node_type', values=node_types)

#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'node_type'

#color_palette = red,green,blue
color_palette = ['#FF0000','#00FF00','#0000FF']

#Choose colors for node and edge highlighting
node_highlight_color = 'white'
edge_highlight_color = 'black'

#Choose a title!
title = 'MITRE Graph Visuals'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [("Name", "@human_name"),("Degree", "@degree")]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object with spring layout
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
network_graph = from_networkx(DG, nx.spring_layout, scale=10, center=(0, 0))
#network_graph = from_networkx(DG, nx.kamada_kawai_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as spectrum of color palette)
minimum_value_color = 0
maximum_value_color = 2
#Set node size and color
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=linear_cmap(color_by_this_attribute, color_palette, minimum_value_color, maximum_value_color))

#Set node highlight colors
network_graph.node_renderer.hover_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)
network_graph.node_renderer.selection_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)


#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

#Set edge highlight colors
network_graph.edge_renderer.selection_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)
network_graph.edge_renderer.hover_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)

#Highlight nodes and edges
network_graph.selection_policy = NodesAndLinkedEdges()
network_graph.inspection_policy = NodesAndLinkedEdges()


#Add network graph to the plot
plot.renderers.append(network_graph)
plot.plot_width = 1100
plot.plot_height = 550
show(plot)
#save(plot, filename=f"{title}.html")

# Visualize single techniques



In [12]:
from bokeh.core.property_mixins import LineJoin
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap
from bokeh.models import EdgesAndLinkedNodes, NodesAndLinkedEdges
import networkx as nx

# display all rows (be careful)
pd.set_option('display.max_rows', None)
# avoid line breaks
pd.set_option('display.expand_frame_repr', False)

# Techniques of pipedream/incontroller
#df_filtered = df[df['target_human'].isin(['Remote Services','Command-Line Interface','Execution through API','Scripting','User Execution',
#                                          'System Firmware','Valid Accounts','Exploitation for Privilege Escalation','Rootkit','Network Sniffing',
#                                          'Remote System Discovery','Remote System Information Discovery','Default Credentials','Lateral Tool Transfer',
#                                          'Program Download','Remote Services','Valid Accounts','Detect Operating Mode','Point & Tag Identification',
#                                          'Program Upload'])]

df_filtered = df[df['target_human'].isin(['Default Credentials','Valid Accounts'])]
print(df_filtered)
DG = nx.from_pandas_edgelist(df_filtered, source='Source', target='Destination',create_using=nx.Graph())

# set the degrees (connections per node)
degrees = dict(nx.degree(DG))
nx.set_node_attributes(DG, name='degree', values=degrees)

#set the human readable attribute
all_mappings = {}
all_mappings.update(mc_technique_mapping)
all_mappings.update(mc_mitigation_mapping)
all_mappings.update(mc_detection_mapping)
nx.set_node_attributes(DG,name='human_name',values=all_mappings)

#ensure very small degrees are still visible
number_to_adjust_by = 5
adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in nx.degree(DG)])
nx.set_node_attributes(DG, name='adjusted_node_size', values=adjusted_node_size)

# determine type, add attribute
node_types = {}
for node in DG.nodes:
  if node.startswith('attack-pattern'):
    node_types[node] = 0
  elif node.startswith('course-of-action'):
    node_types[node] = 1
  else:
    node_types[node] = 2

#set the node type (attack-pattern,course-of-action,data-source)
nx.set_node_attributes(DG, name='node_type', values=node_types)

#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'node_type'

#color_palette = red,green,blue
color_palette = ['#FF0000','#00FF00','#0000FF']

#Choose colors for node and edge highlighting
node_highlight_color = 'white'
edge_highlight_color = 'black'

#Choose a title!
title = 'MITRE Graph Visuals'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [("Name", "@human_name"),("Degree", "@degree")]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object with spring layout
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
network_graph = from_networkx(DG, nx.spring_layout, scale=10, center=(0, 0))
#network_graph = from_networkx(DG, nx.kamada_kawai_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as spectrum of color palette)
minimum_value_color = 0
maximum_value_color = 2
#Set node size and color
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=linear_cmap(color_by_this_attribute, color_palette, minimum_value_color, maximum_value_color))

#Set node highlight colors
network_graph.node_renderer.hover_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)
network_graph.node_renderer.selection_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)


#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

#Set edge highlight colors
network_graph.edge_renderer.selection_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)
network_graph.edge_renderer.hover_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)

#Highlight nodes and edges
network_graph.selection_policy = NodesAndLinkedEdges()
network_graph.inspection_policy = NodesAndLinkedEdges()


#Add network graph to the plot
plot.renderers.append(network_graph)
plot.plot_width = 1100
plot.plot_height = 550
show(plot)
#save(plot, filename=f"{title}.html")

                                                Source                                        Destination                    source_human         target_human
37   x-mitre-data-component--39b9db72-8b48-4595-a18...  attack-pattern--cd2c76a4-5e23-4ca5-9c40-d5e060...          Logon Session Metadata       Valid Accounts
91   x-mitre-data-component--9ce98c86-8d30-4043-ba5...  attack-pattern--cd2c76a4-5e23-4ca5-9c40-d5e060...          Logon Session Creation       Valid Accounts
95   x-mitre-data-component--a953ca55-921a-44f7-9b8...  attack-pattern--cd2c76a4-5e23-4ca5-9c40-d5e060...     User Account Authentication       Valid Accounts
202  x-mitre-data-component--3772e279-27d6-477a-9fe...  attack-pattern--8bb4538f-f16f-49f0-a431-70b544...         Network Traffic Content  Default Credentials
219  x-mitre-data-component--9ce98c86-8d30-4043-ba5...  attack-pattern--8bb4538f-f16f-49f0-a431-70b544...          Logon Session Creation  Default Credentials
278  course-of-action--3992ce42-43e9-4bea-b8db

## ICS visualization testing

After this block we are experimenting with some matching of ICS attacks to their corresponding techniques and mitigations

## Desired relations.

To visualize the desired TTPs the following relationships need to be established.
Campaign has a one-to-many relationship with software.
Every software has a one-to-many relationship with techniques
Every technique has a one-to-many relation with a detection.

Campaign --> Software --> Technique --> Detection

In [13]:
import pandas as pd
from itertools import chain

def get_software(src):
    return list(chain.from_iterable(
        src.query(f) for f in [
            Filter("type", "=", "tool"),
            Filter("type", "=", "malware")
        ]
    ))

MC_SOFTWARE = get_software(MC_DATA)
MC_RELATIONS = MC_DATA.query(Filter("type","=","relationship"))
mc_structured_data = {'Source':[],'Destination':[],'source_human':[],'target_human':[]}
for idx, x in enumerate(MC_RELATIONS):
    if 'relationship_type' in x:
        if x['relationship_type'] == 'detects':
            mc_structured_data['Source'].append(x['source_ref'])
            mc_structured_data['Destination'].append(x['target_ref'])
            mc_structured_data['source_human'].append(mc_detection_mapping[x['source_ref']])
            mc_structured_data['target_human'].append(mc_technique_mapping[x['target_ref']])
# print(MC_RELATIONS)
# print(MC_SOFTWARE)

df = pd.DataFrame(data=mc_structured_data)
df.head()

,Source,Destination,source_human,target_human
0,x-mitre-data-component--3d20385b-24ef-40e1-9f5...,attack-pattern--fa3aa267-da22-4bdd-961f-032233...,Process Creation,Data from Local System
1,x-mitre-data-component--235b7491-2d2b-4617-9a5...,attack-pattern--fa3aa267-da22-4bdd-961f-032233...,File Access,Data from Local System
2,x-mitre-data-component--9bde2f9d-a695-4344-bfa...,attack-pattern--fa3aa267-da22-4bdd-961f-032233...,OS API Execution,Data from Local System
3,x-mitre-data-component--685f917a-e95e-4ba0-ade...,attack-pattern--fa3aa267-da22-4bdd-961f-032233...,Command Execution,Data from Local System
4,x-mitre-data-component--9f387817-df83-432a-b56...,attack-pattern--fa3aa267-da22-4bdd-961f-032233...,Script Execution,Data from Local System


## Helper functions

In [14]:
from pprint import pprint
from stix2 import MemoryStore, Filter

# See section below on "Removing revoked and deprecated objects"
def remove_revoked_deprecated(stix_objects):
    """Remove any revoked or deprecated objects from queries made to the data source"""
    # Note we use .get() because the property may not be present in the JSON data. The default is False
    # if the property is not set.
    return list(
        filter(
            lambda x: x.get("x_mitre_deprecated", False) is False and x.get("revoked", False) is False,
            stix_objects
        )
    )

def get_related(thesrc, src_type, rel_type, target_type, reverse=False):
    """build relationship mappings
       params:
         thesrc: MemoryStore to build relationship lookups for
         src_type: source type for the relationships, e.g "attack-pattern"
         rel_type: relationship type for the relationships, e.g "uses"
         target_type: target type for the relationship, e.g "intrusion-set"
         reverse: build reverse mapping of target to source
    """

    relationships = thesrc.query([
        Filter('type', '=', 'relationship'),
        Filter('relationship_type', '=', rel_type),
        Filter('revoked', '=', False),
    ])

    # See section below on "Removing revoked and deprecated objects"
    relationships = remove_revoked_deprecated(relationships)

    # stix_id => [ { relationship, related_object_id } for each related object ]
    id_to_related = {}

    # build the dict
    for relationship in relationships:
        if src_type in relationship.source_ref and target_type in relationship.target_ref:
            if (relationship.source_ref in id_to_related and not reverse) or (relationship.target_ref in id_to_related and reverse):
                # append to existing entry
                if not reverse:
                    id_to_related[relationship.source_ref].append({
                        "relationship": relationship,
                        "id": relationship.target_ref
                    })
                else:
                    id_to_related[relationship.target_ref].append({
                        "relationship": relationship,
                        "id": relationship.source_ref
                    })
            else:
                # create a new entry
                if not reverse:
                    id_to_related[relationship.source_ref] = [{
                        "relationship": relationship,
                        "id": relationship.target_ref
                    }]
                else:
                    id_to_related[relationship.target_ref] = [{
                        "relationship": relationship,
                        "id": relationship.source_ref
                    }]
    # all objects of relevant type
    if not reverse:
        targets = thesrc.query([
            Filter('type', '=', target_type),
            Filter('revoked', '=', False)
        ])
    else:
        targets = thesrc.query([
            Filter('type', '=', src_type),
            Filter('revoked', '=', False)
        ])

    # build lookup of stixID to stix object
    id_to_target = {}
    for target in targets:
        id_to_target[target.id] = target

    # build final output mappings
    output = {}
    for stix_id in id_to_related:
        value = []
        for related in id_to_related[stix_id]:
            if not related["id"] in id_to_target:
                continue  # targeting a revoked object
            value.append({
                "object": id_to_target[related["id"]],
                "relationship": related["relationship"]
            })
        output[stix_id] = value
    return output


### Get all software used by groups

In [15]:
def software_used_by_groups(thesrc):
    """returns group_id => {software, relationship} for each software used by the group and each software used by campaigns attributed to the group."""
    # get all software used by groups
    tools_used_by_group = get_related(thesrc, "intrusion-set", "uses", "tool")
    malware_used_by_group = get_related(thesrc, "intrusion-set", "uses", "malware")
    software_used_by_group = {**tools_used_by_group, **malware_used_by_group} # group_id -> {software, relationship}

    # get groups attributing to campaigns and all software used by campaigns
    software_used_by_campaign = get_related(thesrc, "campaign", "uses", "tool")
    malware_used_by_campaign = get_related(thesrc, "campaign", "uses", "malware")
    for id in malware_used_by_campaign:
        if id in software_used_by_campaign:
            software_used_by_campaign[id].extend(malware_used_by_campaign[id])
        else:
            software_used_by_campaign[id] = malware_used_by_campaign[id]
    campaigns_attributed_to_group = {
        "campaigns": get_related(thesrc, "campaign", "attributed-to", "intrusion-set", reverse=True), # group_id => {campaign, relationship}
        "software": software_used_by_campaign # campaign_id => {software, relationship}
    }

    for group_id in campaigns_attributed_to_group["campaigns"]:
        software_used_by_campaigns = []
        # check if attributed campaign is using software
        for campaign in campaigns_attributed_to_group["campaigns"][group_id]:
            campaign_id = campaign["object"]["id"]
            if campaign_id in campaigns_attributed_to_group["software"]:
                software_used_by_campaigns.extend(campaigns_attributed_to_group["software"][campaign_id])
        
        # update software used by group to include software used by a groups attributed campaign
        if group_id in software_used_by_group:
            software_used_by_group[group_id].extend(software_used_by_campaigns)
        else:
            software_used_by_group[group_id] = software_used_by_campaigns
    return software_used_by_group

Get

In [17]:
from stix2 import Malware
print(software_used_by_groups(MC_DATA))
print(MC_DATA)

{'intrusion-set--381fcf73-60f6-4ab2-9991-6af3cbc35192': [{'object': Malware(type='malware', id='malware--6a0d0ea9-b2c4-43fe-a552-ac41a3009dc5', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2023-03-30T19:20:45.556Z', modified='2023-04-06T22:00:22.774Z', name='Industroyer2', description='[Industroyer2](https://attack.mitre.org/software/S1072) is a compiled and static piece of malware that has the ability to communicate over the IEC-104 protocol. It is similar to the IEC-104 module found in [Industroyer](https://attack.mitre.org/software/S0604). Security researchers assess that [Industroyer2](https://attack.mitre.org/software/S1072) was designed to cause impact to high-voltage electrical substations. The initial [Industroyer2](https://attack.mitre.org/software/S1072) sample was compiled on 03/23/2022 and scheduled to execute on 04/08/2022, however it was discovered before deploying, resulting in no impact.(Citation: Industroyer2 Blackhat ESET)', revoked=False,

In [21]:
from bokeh.io import output_notebook, show, save
# required to display visuals inline
output_notebook()

Loading BokehJS ...

In [22]:

from bokeh.core.property_mixins import LineJoin
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap
from bokeh.models import EdgesAndLinkedNodes, NodesAndLinkedEdges
import networkx as nx


DG = nx.from_pandas_edgelist(df, source='Source', target='Destination',create_using=nx.Graph())

# set the degrees (connections per node)
degrees = dict(nx.degree(DG))
nx.set_node_attributes(DG, name='degree', values=degrees)

#set the human readable attribute
all_mappings = {}
all_mappings.update(mc_technique_mapping)
all_mappings.update(mc_mitigation_mapping)
all_mappings.update(mc_detection_mapping)
nx.set_node_attributes(DG,name='human_name',values=all_mappings)

#ensure very small degrees are still visible
number_to_adjust_by = 5
adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in nx.degree(DG)])
nx.set_node_attributes(DG, name='adjusted_node_size', values=adjusted_node_size)

# determine type, add attribute
node_types = {}
for node in DG.nodes:
  if node.startswith('x-mitre-data-component'):
    node_types[node] = 0
  elif node.startswith('attack-pattern'):
    node_types[node] = 1
  else:
    node_types[node] = 2

#set the node type (attack-pattern,course-of-action,data-source)
nx.set_node_attributes(DG, name='node_type', values=node_types)

#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'node_type'

#color_palette = red,green,blue
color_palette = ['#FF0000','#00FF00','#0000FF']

#Choose colors for node and edge highlighting
node_highlight_color = 'white'
edge_highlight_color = 'black'

#Choose a title!
title = 'MITRE Graph Visuals'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [("Name", "@human_name"),("Degree", "@degree")]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object with spring layout
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
network_graph = from_networkx(DG, nx.spring_layout, scale=10, center=(0, 0))
#network_graph = from_networkx(DG, nx.kamada_kawai_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as spectrum of color palette)
minimum_value_color = 0
maximum_value_color = 2
#Set node size and color
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=linear_cmap(color_by_this_attribute, color_palette, minimum_value_color, maximum_value_color))

#Set node highlight colors
network_graph.node_renderer.hover_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)
network_graph.node_renderer.selection_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)


#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

#Set edge highlight colors
network_graph.edge_renderer.selection_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)
network_graph.edge_renderer.hover_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)

#Highlight nodes and edges
network_graph.selection_policy = NodesAndLinkedEdges()
network_graph.inspection_policy = NodesAndLinkedEdges()


#Add network graph to the plot
plot.renderers.append(network_graph)
plot.plot_width = 1100
plot.plot_height = 550
show(plot)
#save(plot, filename=f"{title}.html")

# References

* https://melaniewalsh.github.io/Intro-Cultural-Analytics/welcome.html
* Attack CTI
	* https://attackcti.com/intro.html
	* https://github.com/OTRF/ATTACK-Python-Client/blob/f96f9d6094afc657682ccbc1988bae5db39552e9/attackcti/attack_api.py
* MITRE Stix / Taxii
	* https://github.com/mitre/cti/blob/master/USAGE.md
	* https://github.com/mitre-attack/attack-stix-data
	* https://github.com/oasis-open/cti-python-stix2